In [1]:
import nltk
from nltk.corpus import stopwords
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report

In [2]:
# importing the data
df = pd.read_csv('Data.csv')

In [3]:
df.head()

Date  Label                                               Top1  \
0  2000-01-03      0  A 'hindrance to operations': extracts from the...   
1  2000-01-04      0                                          Scorecard   
2  2000-01-05      0                  Coventry caught on counter by Flo   
3  2000-01-06      1                      Pilgrim knows how to progress   
4  2000-01-07      1                               Hitches and Horlocks   

                                 Top2  \
0                           Scorecard   
1                 The best lake scene   
2  United's rivals on the road to Rio   
3                 Thatcher facing ban   
4      Beckham off but United survive   

                                            Top3  \
0                Hughes' instant hit buoys Blues   
1                  Leader: German sleaze inquiry   
2  Thatcher issues defence before trial by video   
3        McIlroy calls for Irish fighting spirit   
4                        Breast cancer screening   

                                            Top4  \
0       Jack gets his skates on at ice-cold Alex   
1                                  Cheerio, boyo   
2  Police help Smith lay down the law at Everton   
3                Leicester bin stadium blueprint   
4                                    Alan Parker   

                                          Top5  \
0       Chaos as Maracana builds up for United   
1                     The main recommendations   
2  Tale of Trautmann bears two more retellings   
3               United braced for Mexican wave   
4      Guardian readers: are you all whingers?   

                                                Top6  \
0  Depleted Leicester prevail as Elliott spoils E...   
1                             Has Cubie killed fees?   
2                                England on the rack   
3  Auntie back in fashion, even if the dress look...   
4                                   Hollywood Beyond   

                                              Top7  \
0                 Hungry Spurs sense rich pickings   
1                           Has Cubie killed fees?   
2  Pakistan retaliate with call for video of Walsh   
3                    Shoaib appeal goes to the top   
4                               Ashes and diamonds   

                                                Top8  ...  \
0                  Gunners so wide of an easy target  ...   
1                             Has Cubie killed fees?  ...   
2               Cullinan continues his Cape monopoly  ...   
3  Hussain hurt by 'shambles' but lays blame on e...  ...   
4                   Whingers - a formidable minority  ...   

                                               Top16  \
0           Flintoff injury piles on woe for England   
1                               On the critical list   
2                        South Melbourne (Australia)   
3  Putin admits Yeltsin quit to give him a head s...   
4                             Most everywhere:  UDIs   

                                               Top17  \
0  Hunters threaten Jospin with new battle of the...   
1                          The timing of their lives   
2                                    Necaxa (Mexico)   
3         BBC worst hit as digital TV begins to bite   
4                       Most wanted:  Chloe lunettes   

                                            Top18  \
0             Kohl's successor drawn into scandal   
1                                     Dear doctor   
2                             Real Madrid (Spain)   
3                     How much can you pay for...   
4  Return of the cane 'completely off the agenda'   

                                               Top19  \
0               The difference between men and women   
1  Irish court halts IRA man's extradition to Nor...   
2                          Raja Casablanca (Morocco)   
3                                 Christmas glitches   
4                   From Sleepy Hollow to Greeneland   

                         

In [4]:
df.shape

(4101, 27)

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4101 entries, 0 to 4100
Data columns (total 27 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Date    4101 non-null   object
 1   Label   4101 non-null   int64 
 2   Top1    4101 non-null   object
 3   Top2    4101 non-null   object
 4   Top3    4101 non-null   object
 5   Top4    4101 non-null   object
 6   Top5    4101 non-null   object
 7   Top6    4101 non-null   object
 8   Top7    4101 non-null   object
 9   Top8    4101 non-null   object
 10  Top9    4101 non-null   object
 11  Top10   4101 non-null   object
 12  Top11   4101 non-null   object
 13  Top12   4101 non-null   object
 14  Top13   4101 non-null   object
 15  Top14   4101 non-null   object
 16  Top15   4101 non-null   object
 17  Top16   4101 non-null   object
 18  Top17   4101 non-null   object
 19  Top18   4101 non-null   object
 20  Top19   4101 non-null   object
 21  Top20   4101 non-null   object
 22  Top21   4101 non-null   

In [6]:
df = df.fillna(' ')

In [7]:
df.isnull().sum()

Date     0
Label    0
Top1     0
Top2     0
Top3     0
Top4     0
Top5     0
Top6     0
Top7     0
Top8     0
Top9     0
Top10    0
Top11    0
Top12    0
Top13    0
Top14    0
Top15    0
Top16    0
Top17    0
Top18    0
Top19    0
Top20    0
Top21    0
Top22    0
Top23    0
Top24    0
Top25    0
dtype: int64

In [8]:
train = df[df['Date'] < '20150101']
test = df[df['Date'] > '20141231']

In [9]:
# removing punctuations
data = train.iloc[:, 2:27]
data.replace("[^a-zA-Z]", " ", regex = True, inplace = True)

In [10]:
# renaming the column for easy use
list1 = [i for i in range(25)]
new_index = [str(i) for i in list1]
data.columns = new_index
data.head()

0  \
0  A  hindrance to operations   extracts from the...   
1                                          Scorecard   
2                  Coventry caught on counter by Flo   
3                      Pilgrim knows how to progress   
4                               Hitches and Horlocks   

                                    1  \
0                           Scorecard   
1                 The best lake scene   
2  United s rivals on the road to Rio   
3                 Thatcher facing ban   
4      Beckham off but United survive   

                                               2  \
0                Hughes  instant hit buoys Blues   
1                  Leader  German sleaze inquiry   
2  Thatcher issues defence before trial by video   
3        McIlroy calls for Irish fighting spirit   
4                        Breast cancer screening   

                                               3  \
0       Jack gets his skates on at ice cold Alex   
1                                  Cheerio  boyo   
2  Police help Smith lay down the law at Everton   
3                Leicester bin stadium blueprint   
4                                    Alan Parker   

                                             4  \
0       Chaos as Maracana builds up for United   
1                     The main recommendations   
2  Tale of Trautmann bears two more retellings   
3               United braced for Mexican wave   
4      Guardian readers  are you all whingers    

                                                   5  \
0  Depleted Leicester prevail as Elliott spoils E...   
1                             Has Cubie killed fees    
2                                England on the rack   
3  Auntie back in fashion  even if the dress look...   
4                                   Hollywood Beyond   

                                                 6  \
0                 Hungry Spurs sense rich pickings   
1                           Has Cubie killed fees    
2  Pakistan retaliate with call for video of Walsh   
3                    Shoaib appeal goes to the top   
4                               Ashes and diamonds   

                                                   7  \
0                  Gunners so wide of an easy target   
1                             Has Cubie killed fees    
2               Cullinan continues his Cape monopoly   
3  Hussain hurt by  shambles  but lays blame on e...   
4                   Whingers   a formidable minority   

                                                   8  \
0      Derby raise a glass to Strupar s debut double   
1  Hopkins  furious  at Foster s lack of Hannibal...   
2             McGrath puts India out of their misery   
3                      England s decade of disasters   
4                             Alan Parker   part two   

                                          9  ...  \
0  Southgate strikes  Leeds pay the penalty  ...   
1                    Has Cubie killed fees   ...   
2            Blair Witch bandwagon rolls on  ...   
3      Revenge is sweet for jubilant Cronje  ...   
4                 Thuggery  Toxins and Ties  ...   

                                                  15  \
0           Flintoff injury piles on woe for England   
1                               On the critical list   
2                        South Melbourne  Australia    
3  Putin admits Yeltsin quit to give him a head s...   
4                             Most everywhere   UDIs   

                                                  16  \
0  Hunters threaten Jospin with new battle of the...   
1                          The timing of their lives   
2                                    Necaxa  Mexico    
3         BBC worst hit as digital TV begins to bite   
4                       Most wanted   Chloe lunettes   

                                               17  \
0             Kohl s successor drawn into scandal   
1                                     Dear doctor   
2                             Real Madrid  Spain    
3    

In [11]:
# convert the dataset into lowercase
for index in new_index:
    data[index] = data[index].str.lower()
data.head(2)

0                    1  \
0  a  hindrance to operations   extracts from the...            scorecard   
1                                          scorecard  the best lake scene   

                                 2                                         3  \
0  hughes  instant hit buoys blues  jack gets his skates on at ice cold alex   
1    leader  german sleaze inquiry                             cheerio  boyo   

                                        4  \
0  chaos as maracana builds up for united   
1                the main recommendations   

                                                   5  \
0  depleted leicester prevail as elliott spoils e...   
1                             has cubie killed fees    

                                  6                                  7  \
0  hungry spurs sense rich pickings  gunners so wide of an easy target   
1            has cubie killed fees              has cubie killed fees    

                                                   8  \
0      derby raise a glass to strupar s debut double   
1  hopkins  furious  at foster s lack of hannibal...   

                                          9  ...  \
0  southgate strikes  leeds pay the penalty  ...   
1                    has cubie killed fees   ...   

                                         15  \
0  flintoff injury piles on woe for england   
1                      on the critical list   

                                                  16  \
0  hunters threaten jospin with new battle of the...   
1                          the timing of their lives   

                                    17  \
0  kohl s successor drawn into scandal   
1                          dear doctor   

                                                  18  \
0               the difference between men and women   
1  irish court halts ira man s extradition to nor...   

                                                  19  \
0                sara denver  nurse turned solicitor   
1  burundi peace initiative fades after rebels re...   

                                               20  \
0  diana s landmine crusade put tories in a panic   
1            pe points the way forward to the ecb   

                                                  21                22  \
0  yeltsin s resignation caught opposition flat f...  russian roulette   
1  campaigners keep up pressure on nazi war crime...    jane ratcliffe   

                                                  23  \
0                                           sold out   
1  yet more things you wouldn t know without the ...   

                             24  
0            recovering a title  
1  millennium bug fails to bite  

[2 rows x 25 columns]

In [12]:
# creating a paragraph using along the data for a row
headlines = []
for row in range(0, len(data.index)):
    headlines.append(' '.join(str(x) for x in data.iloc[row, 0:25]))

In [13]:
headlines[0]

'a  hindrance to operations   extracts from the leaked reports scorecard hughes  instant hit buoys blues jack gets his skates on at ice cold alex chaos as maracana builds up for united depleted leicester prevail as elliott spoils everton s party hungry spurs sense rich pickings gunners so wide of an easy target derby raise a glass to strupar s debut double southgate strikes  leeds pay the penalty hammers hand robson a youthful lesson saints party like it s      wear wolves have turned into lambs stump mike catches testy gough s taunt langer escapes to hit     flintoff injury piles on woe for england hunters threaten jospin with new battle of the somme kohl s successor drawn into scandal the difference between men and women sara denver  nurse turned solicitor diana s landmine crusade put tories in a panic yeltsin s resignation caught opposition flat footed russian roulette sold out recovering a title'

In [14]:
# implementing the bag of words
cv = CountVectorizer(ngram_range = (2, 2))
train_dataset = cv.fit_transform(headlines)

In [15]:
# implementing the Random Forest classifier model
model = RandomForestClassifier(n_estimators = 200, criterion = 'entropy')
model.fit(train_dataset, train['Label'])

RandomForestClassifier(criterion='entropy', n_estimators=200)

for testing data

In [16]:
test_transform = []
for row in range(0, len(test.index)):
    test_transform.append(' '.join(str(x) for x in test.iloc[row, 2:27]))
test_dataset = cv.transform(test_transform)

In [17]:
predictions = model.predict(test_dataset)

In [18]:
# checking for accuracy
matrix = confusion_matrix(test['Label'], predictions)
print(matrix)
report = classification_report(test['Label'], predictions)
print(report)
score = accuracy_score(test['Label'], predictions)
print(score)

[[144  42]
 [  6 186]]
              precision    recall  f1-score   support

           0       0.96      0.77      0.86       186
           1       0.82      0.97      0.89       192

    accuracy                           0.87       378
   macro avg       0.89      0.87      0.87       378
weighted avg       0.89      0.87      0.87       378

0.873015873015873
